In [1]:
import pandas as pd
from utils_common import get_db_engine
import re

# Suppresses the GPU-related warning messages that TensorFlow produces
import warnings
warnings.filterwarnings('ignore')


In [2]:
def drop_summary_rows(df):
# Data contains state-level summaries, identified by the County showing the full State name.
# This removes those rows.
    for (idx, row) in df_states.iterrows():
        state, code = row.loc[['State', 'Code']]
        droprow = df.loc[(df['County'] == state) & (df['State'] == code)].index
        df = df.drop(droprow)
    return df

In [3]:
def percent_to_num(df, pct_column, num_column, pop_column):
    df[num_column] = round((df[pct_column] * df[pop_column] / 100),0)
    df = df.drop(pct_column, axis=1)
    return df

In [4]:
# Redundant or troublesome once it's been rolled into Final Project
try:
    db_engine = get_db_engine()
except Exception as e:
    print(f"\nFailed to connect to database engine.\n", e)


In [5]:
file_states = 'Resources/States.csv'
df_states = pd.read_csv(file_states)
df_states.to_sql(name='states', con=db_engine, if_exists='replace', index=False)
df_states.head()

,State,Code
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [6]:
file_vet_pop = 'Resources/veterans_pop_loc_v3.csv'
df_vet_pop = pd.read_csv(file_vet_pop)
df_vet_pop.to_sql(name='vet_pop_unconditioned', con=db_engine, if_exists='replace', index=False)

In [7]:
df_vet_pop = df_vet_pop.drop('Unnamed: 0', axis=1)

In [8]:
df_vet_pop = drop_summary_rows(df_vet_pop)
df_vet_pop.fillna(0)

,Unnamed: 0.1,State,County,Latitude,Longitude,TotalVets,MaleVets,FemaleVets,WhiteVets,BlackVets,HispanicVets,OtherRaceVets,LessThanHSVets,HighSchOnlyVets,SomeCollegeVets,CollegeDegreeVets,EmployeedVets,UnemployeedVets
0,1,AL,Autauga,32.601011,-86.680736,5272,4599,672,4429,509,294,59,158,1099,1667,2346,4562,187
1,2,AL,Baldwin,32.601011,-86.680736,19663,18118,1544,17819,1054,283,575,710,5438,7211,6302,14733,674
2,3,AL,Barbour,32.601011,-86.680736,1328,1222,104,790,495,14,41,163,529,487,147,660,132
3,4,AL,Bibb,32.601011,-86.680736,1433,1373,59,1133,264,33,0,98,783,319,230,815,0
4,5,AL,Blount,32.601011,-86.680736,3417,3136,281,3221,44,102,102,398,1152,1321,544,2204,249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3143,3193,WY,Sweetwater,43.000325,-107.554567,2749,2600,149,2484,9,241,81,63,893,1373,418,2512,70
3144,3194,WY,Teton,43.000325,-107.554567,999,877,121,974,0,0,25,32,275,152,538,796,0
3145,3195,WY,Uinta,43.000325,-107.554567,1069,990,79,1001,11,13,43,45,311,434,277,797,8
3146,3196,WY,Washakie,43.000325,-107.554567,719,691,26,685,0,33,0,102,259,192,165,615,0


In [9]:
# pct_columns = ['MaleVetsPct', 'FemaleVetsPct', 'WhiteVetsPct',
#                'BlackVetsPct', 'HispanicVetsPct', 'OtherRaceVetsPct', 'LessThanHSVetsPct',
#                'HighSchOnlyVetsPct', 'SomeCollegeVetsPct', 'CollegeDegreeVetsPct',
#                'EmployeedVetsPct', 'UnemployeedVetsPct'] 
# pop_column = 'TotalVets'

# for col in pct_columns:
#     num_col = re.sub('Pct', '', col, count=0, flags=0)
#     df_vet_pop = percent_to_num(df_vet_pop, col, num_col, pop_column)

# # Drop the VetPct. We could derive total population from here, but we're already pulling it
# # in underived from the People table.
# df_vet_pop = df_vet_pop.drop('VeteranPopulationPct', axis=1)

In [10]:
df_vet_pop.head()

,Unnamed: 0.1,State,County,Latitude,Longitude,TotalVets,MaleVets,FemaleVets,WhiteVets,BlackVets,HispanicVets,OtherRaceVets,LessThanHSVets,HighSchOnlyVets,SomeCollegeVets,CollegeDegreeVets,EmployeedVets,UnemployeedVets
0,1,AL,Autauga,32.601011,-86.680736,5272,4599,672,4429,509,294,59,158,1099,1667,2346,4562,187
1,2,AL,Baldwin,32.601011,-86.680736,19663,18118,1544,17819,1054,283,575,710,5438,7211,6302,14733,674
2,3,AL,Barbour,32.601011,-86.680736,1328,1222,104,790,495,14,41,163,529,487,147,660,132
3,4,AL,Bibb,32.601011,-86.680736,1433,1373,59,1133,264,33,0,98,783,319,230,815,0
4,5,AL,Blount,32.601011,-86.680736,3417,3136,281,3221,44,102,102,398,1152,1321,544,2204,249


In [11]:
df_vet_pop.to_sql(name='vet_population', con=db_engine, if_exists='replace', index=False)

In [12]:
file_people = 'Resources/People.csv'
df_people = pd.read_csv(file_people)
df_people.to_sql(name='people_unconditioned', con=db_engine, if_exists='replace', index=False)

In [13]:
df_people = drop_summary_rows(df_people)
df_people.head()

,FIPS,State,County,PopChangeRate1819,PopChangeRate1019,TotalPopEst2019,NetMigrationRate1019,NaturalChangeRate1019,Net_International_Migration_Rate_2010_2019,PopChangeRate0010,...,TotalPopEst2014,TotalPopEst2011,Net_International_Migration_2010_2019,NaturalChange1019,TotalPopEst2015,TotalPopEst2016,TotalPopEst2017,NetMigration1019,TotalPopEst2018,TotalPopEstBase2010
1,1001,AL,Autauga,0.605,2.001,55869,0.686,1.315,-0.029,24.96,...,54893,55227,-16.0,720.0,54864,55243,55390,376.0,55533,54597
2,1003,AL,Baldwin,2.469,21.911,223234,21.001,0.910,0.714,29.80,...,199183,186558,1307.0,1667.0,202939,207601,212521,38455.0,217855,182265
3,1005,AL,Barbour,-0.748,-9.664,24686,-8.797,-0.867,0.161,-5.44,...,26755,27341,44.0,-237.0,26283,25806,25157,-2404.0,24872,27455
4,1007,AL,Bibb,0.121,-2.081,22394,-2.099,0.017,0.525,10.03,...,22553,22745,120.0,4.0,22566,22586,22550,-480.0,22367,22915
5,1009,AL,Blount,0.095,0.784,57826,-0.005,0.790,0.207,12.34,...,57526,57560,119.0,453.0,57526,57494,57787,-3.0,57771,57322


In [14]:
df_reduced_people = df_people[['State','County','TotalPopEst2015','TotalPopEst2016','TotalPopEst2017',
                              'TotalPopEst2018','TotalPopEst2019']]

In [15]:
df_reduced_people.fillna(0)

,State,County,TotalPopEst2015,TotalPopEst2016,TotalPopEst2017,TotalPopEst2018,TotalPopEst2019
1,AL,Autauga,54864,55243,55390,55533,55869
2,AL,Baldwin,202939,207601,212521,217855,223234
3,AL,Barbour,26283,25806,25157,24872,24686
4,AL,Bibb,22566,22586,22550,22367,22394
5,AL,Blount,57526,57494,57787,57771,57826
...,...,...,...,...,...,...,...
3266,PR,Vega Baja,54805,53677,52318,50136,50023
3267,PR,Vieques,8965,8833,8666,8362,8386
3268,PR,Villalba,23662,23091,22452,21436,21372
3269,PR,Yabucoa,35076,34363,33519,32254,32282


In [16]:
df_reduced_people['AveragePop2015to2019'] = df_reduced_people.mean(axis=1)

In [17]:
df_reduced_people.to_sql(name='reduced_people', con=db_engine, if_exists='replace', index=False)